In [ ]:
"""
Utility function for showing images.

Intended to be imported in Jupyter notebooks to display pixmap images.

Invocation: "show_image(item, title)", where item is a PyMuPDF object
which has a "get_pixmap" method, and title is an optional string.

The function executes "item.get_pixmap(dpi=150)" and show the resulting
image.


Dependencies
------------
numpy, matplotlib, pymupdf
"""


def show_image(item, title=""):
    """Display a pixmap.

    Just to display Pixmap image of "item" - ignore the man behind the curtain.

    Args:
        item: any PyMuPDF object having a "get_pixmap" method.
        title: a string to be used as image title

    Generates an RGB Pixmap from item using a constant DPI and using matplotlib
    to show it inline of the notebook.
    """
    DPI = 150  # use this resolution
    import numpy as np
    import matplotlib.pyplot as plt

    # %matplotlib inline
    pix = item.get_pixmap(dpi=DPI)
    img = np.ndarray([pix.h, pix.w, 3], dtype=np.uint8, buffer=pix.samples_mv)
    plt.figure(dpi=DPI)  # set the figure's DPI
    plt.title(title)  # set title of image
    _ = plt.imshow(img, extent=(0, pix.w * 72 / DPI, pix.h * 72 / DPI, 0))

In [ ]:
import pandas as pd  # import pandas
import fitz  # import PyMuPDF
if not hasattr(fitz.Page, "find_tables"):
    raise RuntimeError("This PyMuPDF version does not support the table feature")

In [ ]:
from pathlib import Path
doc = fitz.open(list((Path.cwd().parent / "data" / "statements").glob("*.pdf"))[2])  # open example file
page = doc[0]  # read first page to demo the layout
show_image(page,"First Page Content")

In [ ]:
page.find_tables().tables[0].to_pandas()

In [ ]:
page.get_text()

In [ ]:
print(page.get_text())

In [ ]:
import re

## LaBanquePostale

In [ ]:
from pathlib import Path
from parse_bank_statement import read
import pandas as pd
import fitz
import re

In [ ]:
lbp_files = sorted(list((Path.cwd().parent / "data" / "statements" / "LaBanquePostale").glob("*.pdf")))

In [ ]:
dates = pd.Series(sorted([d for d in map(get_emission_date, lbp_files) if d]))
dates.groupby(dates.dt.year).count()

In [ ]:
content = doc[0].get_text()
content

In [ ]:
import re
re.findall(
    r"\n(?P<date>\d{2}/\d{2})\n(?P<transaction>.*(?!\d{2}/\d{2}))", content, flags=re.DOTALL
)

In [ ]:
import re
re.findall(
    r"\n(?P<header>Date.*?)(?P<ancien_solde>Ancien solde au \d{2}/\d{2}\d{4}\n)?.*(?P<first_table_date>\d{2}/\d{2})\n", content, flags=re.DOTALL
)

In [ ]:
read.Statement(pdf=lbp_files[0])

In [ ]:
read.Statement(pdf=Path("/Users/plfelter/dev/parse-bank-statement/data/statements/LaBanquePostale/RelevÃ© de compte.pdf"))

In [ ]:
lbp_files = sorted(
    list(
        (Path.cwd().parent / "data" / "statements" / "LaBanquePostale").glob(
            "*.pdf"
        )
    )
)

def get_statement(pdf):
    try:
        return read.Statement(pdf=pdf)
    except Exception:
        return None

statements = [s for s in map(get_statement, lbp_files) if s]
print(
    pd.DataFrame(
        {
            "headers": [len(s.headers) for s in statements],
            "filename": [s.pdf.stem for s in statements]
        },
        index=[s.emission_date for s in statements],
    ).sort_index()
)